In [20]:
import os
import re
import h5py

import numpy as np

from tqdm import tqdm

1. Sample preparation
* Generate .root file
* Select the events and save jet constitutent information to .h5 file
    - Sample/from_root_to_h5.py
* Generate mixed sample
    - Training dataset
    - Validation dataset
    - True label testing dataset
* Data augmentation
    - optional
    - apply on training dataset
    - Sample/physical_augmentation_h5.ipynb
* From .h5 file generate jet image and save in .npy file
    - Sample/from_h5_to_npy.py
2. CNN Training

# Sample preparation

## From root to h5

In [19]:
root_path = '../../Software/pythia8307/HVmodel/test_100k-1.root'
output_path = './HVmodel/data/split_val/signal.h5'
sample_type = 1

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

root_path = '../../Software/pythia8307/HVmodel/test_100k-2.root'
output_path = './HVmodel/data/split_val/signal-val.h5'
sample_type = 1

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

root_path = '../../Software/pythia8307/HVmodel/test_100k-3.root'
output_path = './HVmodel/data/split_val/signal-test.h5'
sample_type = 1

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

root_path = './ppjj/Events/run_03/tag_1_delphes_events.root'
output_path = './HVmodel/data/split_val/background_03.h5'
sample_type = 0

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

root_path = './ppjj/Events/run_04/tag_1_delphes_events.root'
output_path = './HVmodel/data/split_val/background_04.h5'
sample_type = 0

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

root_path = './ppjj/Events/run_05/tag_1_delphes_events.root'
output_path = './HVmodel/data/split_val/background-val.h5'
sample_type = 0

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

root_path = './ppjj/Events/run_06/tag_1_delphes_events.root'
output_path = './HVmodel/data/split_val/background-test.h5'
sample_type = 0

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

root_path = './ppjj/Events/run_07/tag_1_delphes_events.root'
output_path = './HVmodel/data/split_val/background_07.h5'
sample_type = 0

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

python from_root_to_h5.py ../../Software/pythia8307/HVmodel/test_100k-1.root ./HVmodel/data/split_val/signal.h5 1 &
python from_root_to_h5.py ../../Software/pythia8307/HVmodel/test_100k-2.root ./HVmodel/data/split_val/signal-val.h5 1 &
python from_root_to_h5.py ../../Software/pythia8307/HVmodel/test_100k-3.root ./HVmodel/data/split_val/signal-test.h5 1 &
python from_root_to_h5.py ./ppjj/Events/run_03/tag_1_delphes_events.root ./HVmodel/data/split_val/background_03.h5 0 &
python from_root_to_h5.py ./ppjj/Events/run_04/tag_1_delphes_events.root ./HVmodel/data/split_val/background_04.h5 0 &
python from_root_to_h5.py ./ppjj/Events/run_05/tag_1_delphes_events.root ./HVmodel/data/split_val/background-val.h5 0 &
python from_root_to_h5.py ./ppjj/Events/run_06/tag_1_delphes_events.root ./HVmodel/data/split_val/background-test.h5 0 &
python from_root_to_h5.py ./ppjj/Events/run_07/tag_1_delphes_events.root ./HVmodel/data/split_val/background_07.h5 0 &


## Generate mixed sample

### Pre-selection 

In [3]:
import ROOT

ROOT.gROOT.ProcessLine('.include /usr/local/Delphes-3.4.2/')
ROOT.gROOT.ProcessLine('.include /usr/local/Delphes-3.4.2/external/')
ROOT.gInterpreter.Declare('#include "/usr/local/Delphes-3.4.2/classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "/usr/local/Delphes-3.4.2/external/ExRootAnalysis/ExRootTreeReader.h"')
ROOT.gInterpreter.Declare('#include "/usr/local/Delphes-3.4.2/external/ExRootAnalysis/ExRootConfReader.h"')
ROOT.gInterpreter.Declare('#include "/usr/local/Delphes-3.4.2/external/ExRootAnalysis/ExRootTask.h"')
ROOT.gSystem.Load("/usr/local/Delphes-3.4.2/install/lib/libDelphes")

Welcome to JupyROOT 6.20/08


0

In [4]:
def Mjets(*arg):
    # arg: list of jets
    # return: invariant mass of jets
    e_tot, px_tot, py_tot, pz_tot = 0, 0, 0, 0
    
    for jet in arg:
        pt, eta, phi, m = jet[0], jet[1], jet[2], jet[3]
        
        px, py, pz = pt*np.cos(phi), pt*np.sin(phi), pt*np.sinh(eta)
        e = np.sqrt(m**2 + px**2 + py**2 + pz**2)
        
        px_tot += px
        py_tot += py
        pz_tot += pz
        e_tot += e
    
    return np.sqrt(e_tot**2 - px_tot**2 - py_tot**2 - pz_tot**2)

In [5]:
def HV_selection(tree):
    # Hidden Valley model selection
    # 1. 2 jets
    # 2. pT > 750 GeV
    # 3. |eta| < 2.0

    SRSB_region = [4400, 4700, 5500, 5800]

    n_event_count = 0
    n_jet_count = 0
    jet_pt_count = 0
    jet_eta_count = 0
    mjjs = []

    pt = [[],[]]

    for event_id, event in tqdm(enumerate(tree)):
        n_event_count += 1

        if event.Jet_size < 2:
            continue
        n_jet_count += 1

        pt[0].append(event.Jet[0].PT)
        pt[1].append(event.Jet[1].PT)
        if event.Jet[1].PT < 750:
            continue
        jet_pt_count += 1

        if abs(event.Jet[0].Eta) > 2.0 or abs(event.Jet[1].Eta) > 2.0:
            continue
        jet_eta_count += 1

        jets = [[event.Jet[i].PT, event.Jet[i].Eta, event.Jet[i].Phi, event.Jet[i].Mass] for i in range(2)]
        mjj = Mjets(*jets)
        mjjs.append(mjj)

        if mjj < SRSB_region[0] or mjj > SRSB_region[3]:
            continue


    mjjs = np.array(mjjs)
    SR_count = ((mjjs > SRSB_region[1]) & (mjjs < SRSB_region[2])).sum()
    SB_count = (((mjjs > SRSB_region[0]) & (mjjs < SRSB_region[1])) | ((mjjs > SRSB_region[2]) & (mjjs < SRSB_region[3]))).sum()

    cutflow_number = {
        'Total': n_event_count,
        'n jet cut': n_jet_count,
        'jet pt cut': jet_pt_count,
        'jet eta cut': jet_eta_count,
        'Signal region': SR_count,
        'Sideband region': SB_count,
    }

    results = {
        'mjj': mjjs,
        'pt': np.array(pt),
        'cutflow_number': cutflow_number,
    }

    return results

In [7]:
root_file = '../Software/pythia8307/HVmodel/test_100k-1.root'
f = ROOT.TFile(root_file)
tree_s = f.Get("Delphes")

results_s = HV_selection(tree_s)

0it [00:00, ?it/s]

100000it [08:58, 185.74it/s]


In [8]:
root_file = './Sample/ppjj/Events/run_02/tag_1_delphes_events.root'
f = ROOT.TFile(root_file)
tree_b = f.Get("Delphes")

results_b = HV_selection(tree_b)

100000it [07:45, 215.05it/s]
Warning in <TStreamerInfo::BuildCheck>: 
   The StreamerInfo for version 2 of class GenParticle read from the file ./Sample/ppjj/Events/run_02/tag_1_delphes_events.root
   has a different checksum than the previously loaded StreamerInfo.
   Reading objects of type GenParticle from the file ./Sample/ppjj/Events/run_02/tag_1_delphes_events.root 
   (and potentially other files) might not work correctly.
   Most likely the version number of the class was not properly
   updated [See ClassDef(GenParticle,2)].
Warning in <TStreamerInfo::CompareContent>: The following data member of
the on-file layout version 2 of class 'GenParticle' differs from 
the in-memory layout version 2:
   float T; //number
vs
   float CtgTheta; //number
Warning in <TStreamerInfo::CompareContent>: The following data member of
the on-file layout version 2 of class 'GenParticle' differs from 
the in-memory layout version 2:
   float X; //number
vs
   float D0; //number
Warning in <TStreame

In [9]:
# modified SB region to 4400-4700, 5500-5800
np.save('./Sample/HVmodel/data/selection_results_SB_4400_5800_s.npy', results_s)
np.save('./Sample/HVmodel/data/selection_results_SB_4400_5800_b.npy', results_b)

### Compute sample size

In [21]:
def get_info(path):
    # path: run path
    name = os.path.split(path)[1]

    with open(os.path.join(path, f'{name}_tag_1_banner.txt')) as f:
        for line in f.readlines():
                
            #  Integrated weight (pb)  :       0.020257
            match = re.match('#  Integrated weight \(pb\)  : +(\d+\.\d+)', line)
            if match:
                # unit: fb
                cross_section = float(match.group(1)) * 1000
            # #  Number of Events        :       100000
            match = re.match('#  Number of Events        :       (\d+)', line)
            if match:
                # unit: fb
                nevent = int(match.group(1))
    
    return cross_section, nevent

def get_dataset_keys(f):
    keys = []
    f.visit(lambda key : keys.append(key) if isinstance(f[key], h5py.Dataset) else None)
    return keys

def create_dataset(f, nevent, MAX_JETS):

    f.create_dataset('J1/MASK', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='|b1')
    f.create_dataset('J1/pt', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J1/eta', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J1/phi', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')

    f.create_dataset('J2/MASK', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='|b1')
    f.create_dataset('J2/pt', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J2/eta', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J2/phi', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')

    f.create_dataset('EVENT/Mjj', (nevent,), maxshape=(None,), dtype='<f4')
    f.create_dataset('EVENT/signal', (nevent,), maxshape=(None,), dtype='<i8')

In [22]:
results_s = np.load('./Sample/HVmodel/data/selection_results_SB_4400_5800_s.npy', allow_pickle=True).item()
results_b = np.load('./Sample/HVmodel/data/selection_results_SB_4400_5800_b.npy', allow_pickle=True).item()

In [23]:
# Total cross section and number of events
xection, tot_event = get_info('./Sample/ppjj/Events/run_03')
print(xection, tot_event)

# cross section in signal region and sideband region
cross_section_SR = results_b['cutflow_number']['Signal region'] / results_b['cutflow_number']['Total'] * xection
cross_section_SB = results_b['cutflow_number']['Sideband region'] / results_b['cutflow_number']['Total'] * xection
print(f'Background cross section, SR: {cross_section_SR:.2f} fb, SB: {cross_section_SB:.2f} fb')

# number of background events in signal region and sideband region
L = 139 * 4
n_SR_B = cross_section_SR * L
n_SB_B = cross_section_SB * L

print(f'Background sample size: SR: {n_SR_B:.1f}, SB: {n_SB_B:.1f}')

sensitivity = 1
n_SR_S = sensitivity * np.sqrt(n_SR_B)
n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']
print(n_SR_S, n_SB_S)

6837.392481 1000000
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 75689.7, SB: 80935.9
275.11754082386454 67.00583183055264


In [11]:
# Training and validation splitting ratio
r_train = 0.8
r_val = 0.2

### Make Training sample

In [26]:
def create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path, SRSB_region=[4400, 4700, 5500, 5800]):
    # n_sig_1: number of signal events in mixing sample 1 (Signal region)
    # n_sig_2: number of signal events in mixing sample 2 (Sideband region)

    nevent = n_sig_1 + n_sig_2 + n_bkg_1 + n_bkg_2
    with h5py.File(output_path, 'w') as f_out:
        MAX_JETS = 300 
        create_dataset(f_out, nevent, MAX_JETS)

        keys = get_dataset_keys(f_out)
        with h5py.File(sig_path, 'r') as f_sig, h5py.File(bkg_path, 'r') as f_bkg:  
            mjj_s = f_sig['EVENT/Mjj'][:]
            mjj_b = f_bkg['EVENT/Mjj'][:]
            SR_range_s = (mjj_s > SRSB_region[1]) & (mjj_s < SRSB_region[2])
            SB_range_s = ((mjj_s > SRSB_region[0]) & (mjj_s < SRSB_region[1])) | ((mjj_s > SRSB_region[2]) & (mjj_s < SRSB_region[3]))
            SR_range_b = (mjj_b > SRSB_region[1]) & (mjj_b < SRSB_region[2])
            SB_range_b = ((mjj_b > SRSB_region[0]) & (mjj_b < SRSB_region[1])) | ((mjj_b > SRSB_region[2]) & (mjj_b < SRSB_region[3]))

            for key in keys:
                f_out[key][:n_sig_1] = f_sig[key][:][SR_range_s][:n_sig_1]
                f_out[key][n_sig_1:n_sig_1+n_bkg_1] = f_bkg[key][:][SR_range_b][:n_bkg_1]
                f_out[key][n_sig_1+n_bkg_1:n_sig_1+n_bkg_1+n_sig_2] = f_sig[key][:][SB_range_s][:n_sig_2]
                f_out[key][n_sig_1+n_bkg_1+n_sig_2:] = f_bkg[key][:][SB_range_b][:n_bkg_2]

        f_out['EVENT/signal'][:n_sig_1+n_bkg_1] = 1
        f_out['EVENT/signal'][n_sig_1+n_bkg_1:] = 0

#### Original mix sample

In [7]:
# new data process flow
# background.h5 merged from background_03.h5 background_04.h5 files
sig_path = './Sample/HVmodel/data/split_val/signal.h5'
bkg_path = './Sample/HVmodel/data/split_val/background.h5'

for i in range(11):
    sensitivity = i
    n_SR_S = sensitivity * np.sqrt(n_SR_B)
    n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']

    n_sig_1, n_sig_2, n_bkg_1, n_bkg_2 = int(n_SR_S * r_train), int(n_SB_S * r_train), int(n_SR_B * r_train), int(n_SB_B * r_train)
    output_path = f'./Sample/HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path)

#### Luminosity $\times 2$

In [12]:
sig_path = './Sample/HVmodel/data/split_val/signal.h5'
bkg_path = './Sample/HVmodel/data/split_val/background.h5'

for i in range(11):
    sensitivity = i
    n_SR_S = sensitivity * np.sqrt(n_SR_B)
    n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']

    n_sig_1, n_sig_2, n_bkg_1, n_bkg_2 = int(n_SR_S * r_train), int(n_SB_S * r_train), int(n_SR_B * r_train), int(n_SB_B * r_train)
    output_path = f'./Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_x2.h5'
    create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path)

In [25]:
sig_path = './Sample/HVmodel/data/split_val/signal.h5'
bkg_path = './Sample/HVmodel/data/split_val/background.h5'

for i in range(1):
    sensitivity = i
    n_SR_S = sensitivity * np.sqrt(n_SR_B)
    n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']

    n_sig_1, n_sig_2, n_bkg_1, n_bkg_2 = int(n_SR_S * r_train), int(n_SB_S * r_train), int(n_SR_B * r_train), int(n_SB_B * r_train)
    output_path = f'./Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_x4.h5'
    create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path)

39548 42189


TypeError: Can't broadcast (39548,) -> (60551,)

### Make Validation sample

In [8]:
sig_path = './Sample/HVmodel/data/split_val/signal-val.h5'
bkg_path = './Sample/HVmodel/data/split_val/background-val.h5'

for i in range(11):
    sensitivity = i
    n_SR_S = sensitivity * np.sqrt(n_SR_B)
    n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']

    n_sig_1, n_sig_2, n_bkg_1, n_bkg_2 = int(n_SR_S * r_val), int(n_SB_S * r_val), int(n_SR_B * r_val), int(n_SB_B * r_val)
    output_path = f'./Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val.h5'
    create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path, SRSB_region=[4400, 4700, 5500, 5800])

#### Luminosity $\times 2$

In [13]:
sig_path = './Sample/HVmodel/data/split_val/signal-val.h5'
bkg_path = './Sample/HVmodel/data/split_val/background-val.h5'

for i in range(11):
    sensitivity = i
    n_SR_S = sensitivity * np.sqrt(n_SR_B)
    n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']

    n_sig_1, n_sig_2, n_bkg_1, n_bkg_2 = int(n_SR_S * r_val), int(n_SB_S * r_val), int(n_SR_B * r_val), int(n_SB_B * r_val)
    output_path = f'./Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_x2_val.h5'
    create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path, SRSB_region=[4400, 4700, 5500, 5800])

### Make Testing sample

In [9]:
sig_path, bkg_path  = './Sample/HVmodel/data/split_val/signal-test.h5', './Sample/HVmodel/data/split_val/background-test.h5'
n_sig_1, n_bkg_1 = 10000, 10000
nevent = n_sig_1 + n_bkg_1

with h5py.File('./Sample/HVmodel/data/split_val/mix_sample_test.h5', 'w') as f_out:
    
    MAX_JETS = 300 
    create_dataset(f_out, nevent, MAX_JETS)

    keys = get_dataset_keys(f_out)
    with h5py.File(sig_path, 'r') as f_sig, h5py.File(bkg_path, 'r') as f_bkg:  
        mjj_s = f_sig['EVENT/Mjj'][:]
        mjj_b = f_bkg['EVENT/Mjj'][:]
        SR_range_s = (mjj_s > 4700) & (mjj_s < 5500)
        SR_range_b = (mjj_b > 4700) & (mjj_b < 5500)

        for key in keys:
            f_out[key][:n_sig_1] = f_sig[key][:][SR_range_s][:n_sig_1]
            f_out[key][n_sig_1:] = f_bkg[key][:][SR_range_b][:n_bkg_1]

    f_out['EVENT/signal'][:n_sig_1] = 1
    f_out['EVENT/signal'][n_sig_1:] = 0

#### Another testing dataset

In [4]:
sig_path, bkg_path  = './Sample/HVmodel/data/new/signal-test-2.h5', './Sample/HVmodel/data/new/background-test-2.h5'
n_sig_1, n_bkg_1 = 10000, 10000
nevent = n_sig_1 + n_bkg_1

with h5py.File('./Sample/HVmodel/data/new/mix_sample_testing-2.h5', 'w') as f_out:
    
    MAX_JETS = 300 
    create_dataset(f_out, nevent, MAX_JETS)

    keys = get_dataset_keys(f_out)
    with h5py.File(sig_path, 'r') as f_sig, h5py.File(bkg_path, 'r') as f_bkg:  
        mjj_s = f_sig['EVENT/Mjj'][:]
        mjj_b = f_bkg['EVENT/Mjj'][:]
        SR_range_s = (mjj_s > 4700) & (mjj_s < 5500)
        SR_range_b = (mjj_b > 4700) & (mjj_b < 5500)

        for key in keys:
            f_out[key][:n_sig_1] = f_sig[key][:][SR_range_s][:n_sig_1]
            f_out[key][n_sig_1:] = f_bkg[key][:][SR_range_b][:n_bkg_1]

    f_out['EVENT/signal'][:n_sig_1] = 1
    f_out['EVENT/signal'][n_sig_1:] = 0

## Data augmentation

Sample/physical_augmentation_h5.ipynb

## From h5 to npy

### Resolution $75 \times 75$

#### Original mix sample

In [3]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/split_val/mix_sample_0.0_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/split_val/mix_sample_1.0_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/split_val/mix_sample_2.0_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/split_val/mix_sample_3.0_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/split_val/mix_sample_4.0_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/split_val/mix_sample_5.0_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0.h5 ./HVmodel/data/split_val/mix_sample_6.0_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_7.0.h5 ./HVmodel/data/split_val/mix_sample_7.0_75x75.npy 75 &
python from_h5_t

#### Validation dataset

In [4]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_val.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_val.h5 ./HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_val.h5 ./HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_val.h5 ./HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_val.h5 ./HVmodel/data/split_val/mix_sample_3.0_val_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_val.h5 ./HVmodel/data/split_val/mix_sample_4.0_val_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_val.h5 ./HVmodel/data/split_val/mix_sample_5.0_val_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_val.h5 ./HVmodel/data/split_val/mix_sample_6.0_val_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_7.0_val.h5 ./HVmodel/d

#### Testing dataset

In [5]:
h5_path = f'./HVmodel/data/split_val/mix_sample_test.h5'
output_path = f'./HVmodel/data/split_val/mix_sample_test_75x75.npy'
resolution = 75

cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_test.h5 ./HVmodel/data/split_val/mix_sample_test_75x75.npy 75 &


#### Copy: +1

In [6]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_copy_1.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_copy_1_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_0.0_copy_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_1.0_copy_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_2.0_copy_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_3.0_copy_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_4.0_copy_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_5.0_copy_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_6.0_copy_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/s

#### Augmentation: +1

In [3]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_aug_1.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_aug_1_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_0.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_1.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_2.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_3.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_4.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_5.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_6.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_s

#### Luminosity $\times 2$

In [10]:
for i in range(11):
    h5_path = f'./HVmodel/data/new/mix_sample_{i:.1f}_x2.h5'
    output_path = f'./HVmodel/data/new/mix_sample_{i:.1f}_x2.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_0.0_x2.h5 ./HVmodel/data/new/mix_sample_0.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_1.0_x2.h5 ./HVmodel/data/new/mix_sample_1.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_2.0_x2.h5 ./HVmodel/data/new/mix_sample_2.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_3.0_x2.h5 ./HVmodel/data/new/mix_sample_3.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_4.0_x2.h5 ./HVmodel/data/new/mix_sample_4.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_5.0_x2.h5 ./HVmodel/data/new/mix_sample_5.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_6.0_x2.h5 ./HVmodel/data/new/mix_sample_6.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_7.0_x2.h5 ./HVmodel/data/new/mix_sample_7.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_8.0_x2.h5 ./HVmodel/data/new/mix_sample_8.0_x2.npy &
python from_h5_to_npy.py ./H

In [7]:
for i in range(11, 21):
    h5_path = f'./HVmodel/data/new/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/new/mix_sample_{i:.1f}.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_11.0.h5 ./HVmodel/data/new/mix_sample_11.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_12.0.h5 ./HVmodel/data/new/mix_sample_12.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_13.0.h5 ./HVmodel/data/new/mix_sample_13.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_14.0.h5 ./HVmodel/data/new/mix_sample_14.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_15.0.h5 ./HVmodel/data/new/mix_sample_15.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_16.0.h5 ./HVmodel/data/new/mix_sample_16.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_17.0.h5 ./HVmodel/data/new/mix_sample_17.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_18.0.h5 ./HVmodel/data/new/mix_sample_18.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_19.0.h5 ./HVmodel/data/new/mix_sample_19.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_20.0.h5 .

In [2]:
for i in range(11):
    h5_path = f'./HVmodel/data/new/mix_sample_{i:.1f}_copy_1.h5'
    output_path = f'./HVmodel/data/new/mix_sample_{i:.1f}_copy_1.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_0.0_copy_1.h5 ./HVmodel/data/new/mix_sample_0.0_copy_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_1.0_copy_1.h5 ./HVmodel/data/new/mix_sample_1.0_copy_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_2.0_copy_1.h5 ./HVmodel/data/new/mix_sample_2.0_copy_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_3.0_copy_1.h5 ./HVmodel/data/new/mix_sample_3.0_copy_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_4.0_copy_1.h5 ./HVmodel/data/new/mix_sample_4.0_copy_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_5.0_copy_1.h5 ./HVmodel/data/new/mix_sample_5.0_copy_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_6.0_copy_1.h5 ./HVmodel/data/new/mix_sample_6.0_copy_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_7.0_copy_1.h5 ./HVmodel/data/new/mix_sample_7.0_copy_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_8.0_copy_1.h5 ./H

#### Only augmentation

In [4]:
for i in range(11):
    h5_path = f'./HVmodel/data/aug_only/mix_sample_{i:.1f}_aug_1.h5'
    output_path = f'./HVmodel/data/aug_only/mix_sample_{i:.1f}_aug_1.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} &'
    print(cmd)

for i in range(11):
    h5_path = f'./HVmodel/data/aug_only/mix_sample_{i:.1f}_aug_2.h5'
    output_path = f'./HVmodel/data/aug_only/mix_sample_{i:.1f}_aug_2.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/aug_only/mix_sample_0.0_aug_1.h5 ./HVmodel/data/aug_only/mix_sample_0.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/aug_only/mix_sample_1.0_aug_1.h5 ./HVmodel/data/aug_only/mix_sample_1.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/aug_only/mix_sample_2.0_aug_1.h5 ./HVmodel/data/aug_only/mix_sample_2.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/aug_only/mix_sample_3.0_aug_1.h5 ./HVmodel/data/aug_only/mix_sample_3.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/aug_only/mix_sample_4.0_aug_1.h5 ./HVmodel/data/aug_only/mix_sample_4.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/aug_only/mix_sample_5.0_aug_1.h5 ./HVmodel/data/aug_only/mix_sample_5.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/aug_only/mix_sample_6.0_aug_1.h5 ./HVmodel/data/aug_only/mix_sample_6.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/aug_only/mix_sample_7.0_aug_1.h5 ./HVmodel/data/aug_only/mix_sample_7.0_aug_1.npy &
python f

#### Smearing scale

In [1]:
for i in range(11):
    h5_path = f'./HVmodel/data/smearing_scale_02/mix_sample_{i:.1f}_aug_1.h5'
    output_path = f'./HVmodel/data/smearing_scale_02/mix_sample_{i:.1f}_aug_1.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_0.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_0.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_1.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_1.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_2.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_2.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_3.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_3.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_4.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_4.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_5.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_5.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_6.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_6.0_aug_1.npy &
python

In [2]:
for i in range(11):
    h5_path = f'./HVmodel/data/smearing_scale_05/mix_sample_{i:.1f}_aug_1.h5'
    output_path = f'./HVmodel/data/smearing_scale_05/mix_sample_{i:.1f}_aug_1.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_0.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_0.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_1.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_1.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_2.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_2.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_3.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_3.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_4.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_4.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_5.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_5.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_6.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_6.0_aug_1.npy &
python

#### Half copy

In [1]:
for i in range(11):
    h5_path = f'./HVmodel/data/new/mix_sample_{i:.1f}_half_copy_1.h5'
    output_path = f'./HVmodel/data/new/mix_sample_{i:.1f}_half_copy_1.h5'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_0.0_half_copy_1.h5 ./HVmodel/data/new/mix_sample_0.0_half_copy_1.h5 &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_1.0_half_copy_1.h5 ./HVmodel/data/new/mix_sample_1.0_half_copy_1.h5 &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_2.0_half_copy_1.h5 ./HVmodel/data/new/mix_sample_2.0_half_copy_1.h5 &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_3.0_half_copy_1.h5 ./HVmodel/data/new/mix_sample_3.0_half_copy_1.h5 &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_4.0_half_copy_1.h5 ./HVmodel/data/new/mix_sample_4.0_half_copy_1.h5 &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_5.0_half_copy_1.h5 ./HVmodel/data/new/mix_sample_5.0_half_copy_1.h5 &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_6.0_half_copy_1.h5 ./HVmodel/data/new/mix_sample_6.0_half_copy_1.h5 &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_7.0_half_copy_1.h5 ./HVmodel/data/new/mix_sample_7.0_half_copy_1.h5 &


### Resolution $25 \times 25$

#### Original mix sample

In [2]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/split_val/mix_sample_0.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/split_val/mix_sample_1.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/split_val/mix_sample_2.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/split_val/mix_sample_3.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/split_val/mix_sample_4.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/split_val/mix_sample_5.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0.h5 ./HVmodel/data/split_val/mix_sample_6.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_7.0.h5 ./HVmodel/data/split_val/mix_sample_7.0_25x25.npy 25 &
python from_h5_t

#### Validation dataset

In [3]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_val.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_val.h5 ./HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_val.h5 ./HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_val.h5 ./HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_val.h5 ./HVmodel/data/split_val/mix_sample_3.0_val_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_val.h5 ./HVmodel/data/split_val/mix_sample_4.0_val_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_val.h5 ./HVmodel/data/split_val/mix_sample_5.0_val_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_val.h5 ./HVmodel/data/split_val/mix_sample_6.0_val_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_7.0_val.h5 ./HVmodel/d

#### Testing dataset

In [4]:
h5_path = f'./HVmodel/data/split_val/mix_sample_test.h5'
output_path = f'./HVmodel/data/split_val/mix_sample_test_25x25.npy'
resolution = 25

cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_test.h5 ./HVmodel/data/split_val/mix_sample_test_25x25.npy 25 &


#### Copy: +1

In [2]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_copy_1.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_copy_1_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_0.0_copy_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_1.0_copy_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_2.0_copy_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_3.0_copy_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_4.0_copy_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_5.0_copy_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_6.0_copy_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/s

#### Luminosity: $\times 2$

In [17]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_x2.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_x2_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)
    
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_x2_val.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_x2_val_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_x2.h5 ./HVmodel/data/split_val/mix_sample_0.0_x2_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_x2.h5 ./HVmodel/data/split_val/mix_sample_1.0_x2_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_x2.h5 ./HVmodel/data/split_val/mix_sample_2.0_x2_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_x2.h5 ./HVmodel/data/split_val/mix_sample_3.0_x2_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_x2.h5 ./HVmodel/data/split_val/mix_sample_4.0_x2_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_x2.h5 ./HVmodel/data/split_val/mix_sample_5.0_x2_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_x2.h5 ./HVmodel/data/split_val/mix_sample_6.0_x2_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_7.0_x2.h5 ./HVmodel/data/split_val/m

#### Augmentation: +1, +3

In [4]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_aug_1.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_aug_1_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_0.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_1.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_2.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_3.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_4.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_5.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_6.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_s

In [4]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_aug_3.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_aug_3_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_aug_3.h5 ./HVmodel/data/split_val/mix_sample_0.0_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_aug_3.h5 ./HVmodel/data/split_val/mix_sample_1.0_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_aug_3.h5 ./HVmodel/data/split_val/mix_sample_2.0_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_aug_3.h5 ./HVmodel/data/split_val/mix_sample_3.0_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_aug_3.h5 ./HVmodel/data/split_val/mix_sample_4.0_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_aug_3.h5 ./HVmodel/data/split_val/mix_sample_5.0_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_aug_3.h5 ./HVmodel/data/split_val/mix_sample_6.0_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_s

### Modified SB region

In [22]:
for i in range(11):
    h5_path = f'./HVmodel/data/new_SB/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/new_SB/mix_sample_{i:.1f}_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_0.0.h5 ./HVmodel/data/new_SB/mix_sample_0.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_1.0.h5 ./HVmodel/data/new_SB/mix_sample_1.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_2.0.h5 ./HVmodel/data/new_SB/mix_sample_2.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_3.0.h5 ./HVmodel/data/new_SB/mix_sample_3.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_4.0.h5 ./HVmodel/data/new_SB/mix_sample_4.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_5.0.h5 ./HVmodel/data/new_SB/mix_sample_5.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_6.0.h5 ./HVmodel/data/new_SB/mix_sample_6.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_7.0.h5 ./HVmodel/data/new_SB/mix_sample_7.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_8.0.h5

In [23]:
for i in range(11):
    h5_path = f'./HVmodel/data/new_SB/mix_sample_{i:.1f}_copy_1.h5'
    output_path = f'./HVmodel/data/new_SB/mix_sample_{i:.1f}_25x25_copy_1.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_0.0_copy_1.h5 ./HVmodel/data/new_SB/mix_sample_0.0_25x25_copy_1.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_1.0_copy_1.h5 ./HVmodel/data/new_SB/mix_sample_1.0_25x25_copy_1.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_2.0_copy_1.h5 ./HVmodel/data/new_SB/mix_sample_2.0_25x25_copy_1.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_3.0_copy_1.h5 ./HVmodel/data/new_SB/mix_sample_3.0_25x25_copy_1.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_4.0_copy_1.h5 ./HVmodel/data/new_SB/mix_sample_4.0_25x25_copy_1.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_5.0_copy_1.h5 ./HVmodel/data/new_SB/mix_sample_5.0_25x25_copy_1.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_6.0_copy_1.h5 ./HVmodel/data/new_SB/mix_sample_6.0_25x25_copy_1.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/new_SB/mix_sample_7.0_copy_1.h5 ./HVmodel/d

# CNN Training

In [11]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}.npy'
    model_name = f'SB_{i:.1f}_new'
    sample_type = f'Sensitivity: {i:.1f}'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0.npy SB_0.0_new "Sensitivity: 0.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0.npy SB_1.0_new "Sensitivity: 1.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0.npy SB_2.0_new "Sensitivity: 2.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0.npy SB_3.0_new "Sensitivity: 3.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0.npy SB_4.0_new "Sensitivity: 4.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0.npy SB_5.0_new "Sensitivity: 5.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0.npy SB_6.0_new "Sensitivity: 6.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0.npy SB_7.0_new "Sensitivity: 7.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_8.0.npy SB_8.0_new "Sensitivity: 8.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_9.0.npy SB_9.0_new "Sensitivity: 9.0"
python train_CNN.py ../Sample/HVmodel/da

In [12]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_aug_1.npy'
    model_name = f'SB_{i:.1f}_aug_1_new'
    sample_type = f'Sensitivity: {i:.1f}, Augmentation: 1'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_aug_1.npy SB_0.0_aug_1_new "Sensitivity: 0.0, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_aug_1.npy SB_1.0_aug_1_new "Sensitivity: 1.0, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_aug_1.npy SB_2.0_aug_1_new "Sensitivity: 2.0, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_aug_1.npy SB_3.0_aug_1_new "Sensitivity: 3.0, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_aug_1.npy SB_4.0_aug_1_new "Sensitivity: 4.0, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_aug_1.npy SB_5.0_aug_1_new "Sensitivity: 5.0, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_aug_1.npy SB_6.0_aug_1_new "Sensitivity: 6.0, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_aug_1.npy SB_7.0_aug_1_new "Sensitivity: 7.0, Augmentation: 1"


In [13]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_x2.npy'
    model_name = f'SB_{i:.1f}_x2_new'
    sample_type = f'Sensitivity: {i:.1f}, Luminosity: x2'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_x2.npy SB_0.0_x2_new "Sensitivity: 0.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_x2.npy SB_1.0_x2_new "Sensitivity: 1.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_x2.npy SB_2.0_x2_new "Sensitivity: 2.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_x2.npy SB_3.0_x2_new "Sensitivity: 3.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_x2.npy SB_4.0_x2_new "Sensitivity: 4.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_x2.npy SB_5.0_x2_new "Sensitivity: 5.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_x2.npy SB_6.0_x2_new "Sensitivity: 6.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_x2.npy SB_7.0_x2_new "Sensitivity: 7.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sampl

In [8]:
for i in range(11, 21):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}.npy'
    model_name = f'SB_{i:.1f}_new'
    sample_type = f'Sensitivity: {i:.1f}'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_11.0.npy SB_11.0_new "Sensitivity: 11.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_12.0.npy SB_12.0_new "Sensitivity: 12.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_13.0.npy SB_13.0_new "Sensitivity: 13.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_14.0.npy SB_14.0_new "Sensitivity: 14.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_15.0.npy SB_15.0_new "Sensitivity: 15.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_16.0.npy SB_16.0_new "Sensitivity: 16.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_17.0.npy SB_17.0_new "Sensitivity: 17.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_18.0.npy SB_18.0_new "Sensitivity: 18.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_19.0.npy SB_19.0_new "Sensitivity: 19.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_20.0.npy SB_20.0_new "Sensitivity: 20.0"


In [3]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_copy_1.npy'
    model_name = f'SB_{i:.1f}_copy_1_new'
    sample_type = f'Sensitivity: {i:.1f}'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_copy_1.npy SB_0.0_copy_1_new "Sensitivity: 0.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_copy_1.npy SB_1.0_copy_1_new "Sensitivity: 1.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_copy_1.npy SB_2.0_copy_1_new "Sensitivity: 2.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_copy_1.npy SB_3.0_copy_1_new "Sensitivity: 3.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_copy_1.npy SB_4.0_copy_1_new "Sensitivity: 4.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_copy_1.npy SB_5.0_copy_1_new "Sensitivity: 5.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_copy_1.npy SB_6.0_copy_1_new "Sensitivity: 6.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_copy_1.npy SB_7.0_copy_1_new "Sensitivity: 7.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_8.0_copy_1.npy SB_8.0_copy_1_new "Sensitivity: 8.0"
python tra

In [5]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/aug_only/mix_sample_{i:.1f}_aug_1.npy'
    model_name = f'SB_{i:.1f}_only_aug_1_new'
    sample_type = f'Sensitivity: {i:.1f}, Only Augmentation: 1'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/aug_only/mix_sample_0.0_aug_1.npy SB_0.0_only_aug_1_new "Sensitivity: 0.0, Only Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/aug_only/mix_sample_1.0_aug_1.npy SB_1.0_only_aug_1_new "Sensitivity: 1.0, Only Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/aug_only/mix_sample_2.0_aug_1.npy SB_2.0_only_aug_1_new "Sensitivity: 2.0, Only Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/aug_only/mix_sample_3.0_aug_1.npy SB_3.0_only_aug_1_new "Sensitivity: 3.0, Only Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/aug_only/mix_sample_4.0_aug_1.npy SB_4.0_only_aug_1_new "Sensitivity: 4.0, Only Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/aug_only/mix_sample_5.0_aug_1.npy SB_5.0_only_aug_1_new "Sensitivity: 5.0, Only Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/aug_only/mix_sample_6.0_aug_1.npy SB_6.0_only_aug_1_new "Sensitivity: 6.0, Only Augmentation: 1"
python train_CNN.py 

In [6]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/aug_only/mix_sample_{i:.1f}_aug_2.npy'
    model_name = f'SB_{i:.1f}_only_aug_2_new'
    sample_type = f'Sensitivity: {i:.1f}, Only Augmentation: 2'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/aug_only/mix_sample_0.0_aug_2.npy SB_0.0_only_aug_2_new "Sensitivity: 0.0, Only Augmentation: 2"
python train_CNN.py ../Sample/HVmodel/data/aug_only/mix_sample_1.0_aug_2.npy SB_1.0_only_aug_2_new "Sensitivity: 1.0, Only Augmentation: 2"
python train_CNN.py ../Sample/HVmodel/data/aug_only/mix_sample_2.0_aug_2.npy SB_2.0_only_aug_2_new "Sensitivity: 2.0, Only Augmentation: 2"
python train_CNN.py ../Sample/HVmodel/data/aug_only/mix_sample_3.0_aug_2.npy SB_3.0_only_aug_2_new "Sensitivity: 3.0, Only Augmentation: 2"
python train_CNN.py ../Sample/HVmodel/data/aug_only/mix_sample_4.0_aug_2.npy SB_4.0_only_aug_2_new "Sensitivity: 4.0, Only Augmentation: 2"
python train_CNN.py ../Sample/HVmodel/data/aug_only/mix_sample_5.0_aug_2.npy SB_5.0_only_aug_2_new "Sensitivity: 5.0, Only Augmentation: 2"
python train_CNN.py ../Sample/HVmodel/data/aug_only/mix_sample_6.0_aug_2.npy SB_6.0_only_aug_2_new "Sensitivity: 6.0, Only Augmentation: 2"
python train_CNN.py 

## Change smearing scale

In [2]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/smearing_scale_02/mix_sample_{i:.1f}_aug_1.npy'
    model_name = f'SB_{i:.1f}_aug_1_std_02_new'
    sample_type = f'Sensitivity: {i:.1f}, Augmentation: 1, Smearing: 0.2'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/smearing_scale_02/mix_sample_0.0_aug_1.npy SB_0.0_aug_1_std_02_new "Sensitivity: 0.0, Augmentation: 1, Smearing: 0.2"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_02/mix_sample_1.0_aug_1.npy SB_1.0_aug_1_std_02_new "Sensitivity: 1.0, Augmentation: 1, Smearing: 0.2"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_02/mix_sample_2.0_aug_1.npy SB_2.0_aug_1_std_02_new "Sensitivity: 2.0, Augmentation: 1, Smearing: 0.2"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_02/mix_sample_3.0_aug_1.npy SB_3.0_aug_1_std_02_new "Sensitivity: 3.0, Augmentation: 1, Smearing: 0.2"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_02/mix_sample_4.0_aug_1.npy SB_4.0_aug_1_std_02_new "Sensitivity: 4.0, Augmentation: 1, Smearing: 0.2"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_02/mix_sample_5.0_aug_1.npy SB_5.0_aug_1_std_02_new "Sensitivity: 5.0, Augmentation: 1, Smearing: 0.2"
python train_CNN.py ../Sample/HVmo

In [3]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/smearing_scale_05/mix_sample_{i:.1f}_aug_1.npy'
    model_name = f'SB_{i:.1f}_aug_1_std_05_new'
    sample_type = f'Sensitivity: {i:.1f}, Augmentation: 1, Smearing: 0.5'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/smearing_scale_05/mix_sample_0.0_aug_1.npy SB_0.0_aug_1_std_05_new "Sensitivity: 0.0, Augmentation: 1, Smearing: 0.5"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_05/mix_sample_1.0_aug_1.npy SB_1.0_aug_1_std_05_new "Sensitivity: 1.0, Augmentation: 1, Smearing: 0.5"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_05/mix_sample_2.0_aug_1.npy SB_2.0_aug_1_std_05_new "Sensitivity: 2.0, Augmentation: 1, Smearing: 0.5"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_05/mix_sample_3.0_aug_1.npy SB_3.0_aug_1_std_05_new "Sensitivity: 3.0, Augmentation: 1, Smearing: 0.5"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_05/mix_sample_4.0_aug_1.npy SB_4.0_aug_1_std_05_new "Sensitivity: 4.0, Augmentation: 1, Smearing: 0.5"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_05/mix_sample_5.0_aug_1.npy SB_5.0_aug_1_std_05_new "Sensitivity: 5.0, Augmentation: 1, Smearing: 0.5"
python train_CNN.py ../Sample/HVmo

## Remove shuffle

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_copy_1_no_shuffle.npy'
    model_name = f'SB_{i:.1f}_copy_1_no_shuffle_new'
    sample_type = f'Sensitivity: {i:.1f}, Copy: 1, No shuffle'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_copy_1_no_shuffle.npy SB_0.0_copy_1_no_shuffle_new "Sensitivity: 0.0, Copy: 1, No shuffle"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_copy_1_no_shuffle.npy SB_1.0_copy_1_no_shuffle_new "Sensitivity: 1.0, Copy: 1, No shuffle"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_copy_1_no_shuffle.npy SB_2.0_copy_1_no_shuffle_new "Sensitivity: 2.0, Copy: 1, No shuffle"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_copy_1_no_shuffle.npy SB_3.0_copy_1_no_shuffle_new "Sensitivity: 3.0, Copy: 1, No shuffle"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_copy_1_no_shuffle.npy SB_4.0_copy_1_no_shuffle_new "Sensitivity: 4.0, Copy: 1, No shuffle"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_copy_1_no_shuffle.npy SB_5.0_copy_1_no_shuffle_new "Sensitivity: 5.0, Copy: 1, No shuffle"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_copy_1_no_shuffle.np

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_copy_1_no_shuffle.npy'
    model_name = f'SB_{i:.1f}_copy_1_no_shuffle_code'
    sample_type = f'Sensitivity: {i:.1f}, Copy: 1, No shuffle'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_copy_1_no_shuffle.npy SB_0.0_copy_1_no_shuffle_code "Sensitivity: 0.0, Copy: 1, No shuffle"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_copy_1_no_shuffle.npy SB_1.0_copy_1_no_shuffle_code "Sensitivity: 1.0, Copy: 1, No shuffle"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_copy_1_no_shuffle.npy SB_2.0_copy_1_no_shuffle_code "Sensitivity: 2.0, Copy: 1, No shuffle"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_copy_1_no_shuffle.npy SB_3.0_copy_1_no_shuffle_code "Sensitivity: 3.0, Copy: 1, No shuffle"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_copy_1_no_shuffle.npy SB_4.0_copy_1_no_shuffle_code "Sensitivity: 4.0, Copy: 1, No shuffle"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_copy_1_no_shuffle.npy SB_5.0_copy_1_no_shuffle_code "Sensitivity: 5.0, Copy: 1, No shuffle"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_copy_1_no_shuf

## Half + Copy

In [2]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_half_copy_1.npy'
    model_name = f'SB_{i:.1f}_half_copy_1'
    sample_type = f'Sensitivity: {i:.1f}, Half, Copy: 1'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_half_copy_1.npy SB_0.0_half_copy_1 "Sensitivity: 0.0, Half, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_half_copy_1.npy SB_1.0_half_copy_1 "Sensitivity: 1.0, Half, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_half_copy_1.npy SB_2.0_half_copy_1 "Sensitivity: 2.0, Half, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_half_copy_1.npy SB_3.0_half_copy_1 "Sensitivity: 3.0, Half, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_half_copy_1.npy SB_4.0_half_copy_1 "Sensitivity: 4.0, Half, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_half_copy_1.npy SB_5.0_half_copy_1 "Sensitivity: 5.0, Half, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_half_copy_1.npy SB_6.0_half_copy_1 "Sensitivity: 6.0, Half, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_half_copy_1.npy SB_7.

## Orginal + $x$ Copy 

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_copy_0.5.npy'
    model_name = f'SB_{i:.1f}_copy_0.5'
    sample_type = f'Sensitivity: {i:.1f}, Copy: 0.5'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_copy_0.5.npy SB_0.0_copy_0.5 "Sensitivity: 0.0, Copy: 0.5"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_copy_0.5.npy SB_1.0_copy_0.5 "Sensitivity: 1.0, Copy: 0.5"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_copy_0.5.npy SB_2.0_copy_0.5 "Sensitivity: 2.0, Copy: 0.5"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_copy_0.5.npy SB_3.0_copy_0.5 "Sensitivity: 3.0, Copy: 0.5"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_copy_0.5.npy SB_4.0_copy_0.5 "Sensitivity: 4.0, Copy: 0.5"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_copy_0.5.npy SB_5.0_copy_0.5 "Sensitivity: 5.0, Copy: 0.5"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_copy_0.5.npy SB_6.0_copy_0.5 "Sensitivity: 6.0, Copy: 0.5"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_copy_0.5.npy SB_7.0_copy_0.5 "Sensitivity: 7.0, Copy: 0.5"
python train_CNN.py ../Sample/HV

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_copy_0.25.npy'
    model_name = f'SB_{i:.1f}_copy_0.25'
    sample_type = f'Sensitivity: {i:.1f}, Copy: 0.25'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_copy_0.25.npy SB_0.0_copy_0.25 "Sensitivity: 0.0, Copy: 0.25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_copy_0.25.npy SB_1.0_copy_0.25 "Sensitivity: 1.0, Copy: 0.25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_copy_0.25.npy SB_2.0_copy_0.25 "Sensitivity: 2.0, Copy: 0.25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_copy_0.25.npy SB_3.0_copy_0.25 "Sensitivity: 3.0, Copy: 0.25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_copy_0.25.npy SB_4.0_copy_0.25 "Sensitivity: 4.0, Copy: 0.25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_copy_0.25.npy SB_5.0_copy_0.25 "Sensitivity: 5.0, Copy: 0.25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_copy_0.25.npy SB_6.0_copy_0.25 "Sensitivity: 6.0, Copy: 0.25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_copy_0.25.npy SB_7.0_copy_0.25 "Sensitivity: 7.0, Copy: 0.25"
python t

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_copy_0.75.npy'
    model_name = f'SB_{i:.1f}_copy_0.75'
    sample_type = f'Sensitivity: {i:.1f}, Copy: 0.75'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_copy_0.75.npy SB_0.0_copy_0.75 "Sensitivity: 0.0, Copy: 0.75"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_copy_0.75.npy SB_1.0_copy_0.75 "Sensitivity: 1.0, Copy: 0.75"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_copy_0.75.npy SB_2.0_copy_0.75 "Sensitivity: 2.0, Copy: 0.75"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_copy_0.75.npy SB_3.0_copy_0.75 "Sensitivity: 3.0, Copy: 0.75"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_copy_0.75.npy SB_4.0_copy_0.75 "Sensitivity: 4.0, Copy: 0.75"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_copy_0.75.npy SB_5.0_copy_0.75 "Sensitivity: 5.0, Copy: 0.75"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_copy_0.75.npy SB_6.0_copy_0.75 "Sensitivity: 6.0, Copy: 0.75"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_copy_0.75.npy SB_7.0_copy_0.75 "Sensitivity: 7.0, Copy: 0.75"
python t

## Remove the swapping label procedure

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}.npy'
    model_name = f'SB_{i:.1f}_no_swap'
    sample_type = f'Sensitivity: {i:.1f}'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0.npy SB_0.0_no_swap "Sensitivity: 0.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0.npy SB_1.0_no_swap "Sensitivity: 1.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0.npy SB_2.0_no_swap "Sensitivity: 2.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0.npy SB_3.0_no_swap "Sensitivity: 3.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0.npy SB_4.0_no_swap "Sensitivity: 4.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0.npy SB_5.0_no_swap "Sensitivity: 5.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0.npy SB_6.0_no_swap "Sensitivity: 6.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0.npy SB_7.0_no_swap "Sensitivity: 7.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_8.0.npy SB_8.0_no_swap "Sensitivity: 8.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_9.0.npy SB_9.0_no_swap "Sensitivity: 9.0"


In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_x2.npy'
    model_name = f'SB_{i:.1f}_x2_no_swap'
    sample_type = f'Sensitivity: {i:.1f}, Luminosity: x2'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_x2.npy SB_0.0_x2_no_swap "Sensitivity: 0.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_x2.npy SB_1.0_x2_no_swap "Sensitivity: 1.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_x2.npy SB_2.0_x2_no_swap "Sensitivity: 2.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_x2.npy SB_3.0_x2_no_swap "Sensitivity: 3.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_x2.npy SB_4.0_x2_no_swap "Sensitivity: 4.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_x2.npy SB_5.0_x2_no_swap "Sensitivity: 5.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_x2.npy SB_6.0_x2_no_swap "Sensitivity: 6.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_x2.npy SB_7.0_x2_no_swap "Sensitivity: 7.0, Luminosity: x2"
python train_CNN.py ../S

In [2]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_copy_1.npy'
    model_name = f'SB_{i:.1f}_copy_1_no_swap'
    sample_type = f'Sensitivity: {i:.1f}, Copy: 1'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_copy_1.npy SB_0.0_copy_1_no_swap "Sensitivity: 0.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_copy_1.npy SB_1.0_copy_1_no_swap "Sensitivity: 1.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_copy_1.npy SB_2.0_copy_1_no_swap "Sensitivity: 2.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_copy_1.npy SB_3.0_copy_1_no_swap "Sensitivity: 3.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_copy_1.npy SB_4.0_copy_1_no_swap "Sensitivity: 4.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_copy_1.npy SB_5.0_copy_1_no_swap "Sensitivity: 5.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_copy_1.npy SB_6.0_copy_1_no_swap "Sensitivity: 6.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_copy_1.npy SB_7.0_copy_1_no_swap "Sensitivity: 7.0, Copy: 1"
python train_CNN

## Change model structure

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}.npy'
    model_name = f'SB_{i:.1f}_2_image'
    sample_type = f'Sensitivity: {i:.1f}'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0.npy SB_0.0_2_image "Sensitivity: 0.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0.npy SB_1.0_2_image "Sensitivity: 1.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0.npy SB_2.0_2_image "Sensitivity: 2.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0.npy SB_3.0_2_image "Sensitivity: 3.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0.npy SB_4.0_2_image "Sensitivity: 4.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0.npy SB_5.0_2_image "Sensitivity: 5.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0.npy SB_6.0_2_image "Sensitivity: 6.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0.npy SB_7.0_2_image "Sensitivity: 7.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_8.0.npy SB_8.0_2_image "Sensitivity: 8.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_9.0.npy SB_9.0_2_image "Sensitivity: 9.0"


In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_copy_1.npy'
    model_name = f'SB_{i:.1f}_copy_1_2_image'
    sample_type = f'Sensitivity: {i:.1f}, Copy: 1'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_copy_1.npy SB_0.0_copy_1_2_image "Sensitivity: 0.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_copy_1.npy SB_1.0_copy_1_2_image "Sensitivity: 1.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_copy_1.npy SB_2.0_copy_1_2_image "Sensitivity: 2.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_copy_1.npy SB_3.0_copy_1_2_image "Sensitivity: 3.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_copy_1.npy SB_4.0_copy_1_2_image "Sensitivity: 4.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_copy_1.npy SB_5.0_copy_1_2_image "Sensitivity: 5.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_copy_1.npy SB_6.0_copy_1_2_image "Sensitivity: 6.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_copy_1.npy SB_7.0_copy_1_2_image "Sensitivity: 7.0, Copy: 1"
python train_CNN

## Tensorflow version

In [2]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}.npy'
    model_name = f'SB_{i:.1f}_2_image_tf2.5'
    sample_type = f'Sensitivity: {i:.1f}'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0.npy SB_0.0_2_image_tf2.5 "Sensitivity: 0.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0.npy SB_1.0_2_image_tf2.5 "Sensitivity: 1.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0.npy SB_2.0_2_image_tf2.5 "Sensitivity: 2.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0.npy SB_3.0_2_image_tf2.5 "Sensitivity: 3.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0.npy SB_4.0_2_image_tf2.5 "Sensitivity: 4.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0.npy SB_5.0_2_image_tf2.5 "Sensitivity: 5.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0.npy SB_6.0_2_image_tf2.5 "Sensitivity: 6.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0.npy SB_7.0_2_image_tf2.5 "Sensitivity: 7.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_8.0.npy SB_8.0_2_image_tf2.5 "Sensitivity: 8.0"
python train_CNN.py ../Sample/HVmodel/data/new

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_copy_1.npy'
    model_name = f'SB_{i:.1f}_copy_1_2_image_tf2.0'
    sample_type = f'Sensitivity: {i:.1f}, Copy: 1'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_copy_1.npy SB_0.0_copy_1_2_image_tf2.0 "Sensitivity: 0.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_copy_1.npy SB_1.0_copy_1_2_image_tf2.0 "Sensitivity: 1.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_copy_1.npy SB_2.0_copy_1_2_image_tf2.0 "Sensitivity: 2.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_copy_1.npy SB_3.0_copy_1_2_image_tf2.0 "Sensitivity: 3.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_copy_1.npy SB_4.0_copy_1_2_image_tf2.0 "Sensitivity: 4.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_copy_1.npy SB_5.0_copy_1_2_image_tf2.0 "Sensitivity: 5.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_copy_1.npy SB_6.0_copy_1_2_image_tf2.0 "Sensitivity: 6.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_copy_1.npy SB_7.0_copy_1_2_image_tf

## Early stopping patience

In [2]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}.npy'
    model_name = f'SB_{i:.1f}_2_image_patience_30'
    sample_type = f'Sensitivity: {i:.1f}'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0.npy SB_0.0_2_image_patience_30 "Sensitivity: 0.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0.npy SB_1.0_2_image_patience_30 "Sensitivity: 1.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0.npy SB_2.0_2_image_patience_30 "Sensitivity: 2.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0.npy SB_3.0_2_image_patience_30 "Sensitivity: 3.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0.npy SB_4.0_2_image_patience_30 "Sensitivity: 4.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0.npy SB_5.0_2_image_patience_30 "Sensitivity: 5.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0.npy SB_6.0_2_image_patience_30 "Sensitivity: 6.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0.npy SB_7.0_2_image_patience_30 "Sensitivity: 7.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_8.0.npy SB_8.0_2_image_patience_30 "Sensitivit

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_copy_1.npy'
    model_name = f'SB_{i:.1f}_copy_1_2_image_patience_30'
    sample_type = f'Sensitivity: {i:.1f}, Copy: 1'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_copy_1.npy SB_0.0_copy_1_2_image_patience_30 "Sensitivity: 0.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_copy_1.npy SB_1.0_copy_1_2_image_patience_30 "Sensitivity: 1.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_copy_1.npy SB_2.0_copy_1_2_image_patience_30 "Sensitivity: 2.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_copy_1.npy SB_3.0_copy_1_2_image_patience_30 "Sensitivity: 3.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_copy_1.npy SB_4.0_copy_1_2_image_patience_30 "Sensitivity: 4.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_copy_1.npy SB_5.0_copy_1_2_image_patience_30 "Sensitivity: 5.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_copy_1.npy SB_6.0_copy_1_2_image_patience_30 "Sensitivity: 6.0, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_samp

## Resolution $25 \times 25$

In [6]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_25x25.npy'
    model_name = f'SB_{i:.1f}_2_image_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_25x25.npy SB_0.0_2_image_25x25 "Sensitivity: 0.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_25x25.npy SB_1.0_2_image_25x25 "Sensitivity: 1.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_25x25.npy SB_2.0_2_image_25x25 "Sensitivity: 2.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_25x25.npy SB_3.0_2_image_25x25 "Sensitivity: 3.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_25x25.npy SB_4.0_2_image_25x25 "Sensitivity: 4.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_25x25.npy SB_5.0_2_image_25x25 "Sensitivity: 5.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_25x25.npy SB_6.0_2_image_25x25 "Sensitivity: 6.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_25x25.npy SB_7.0_2_im

In [3]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_25x25_copy_1.npy'
    model_name = f'SB_{i:.1f}_copy_1_2_image_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Copy: 1'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_25x25_copy_1.npy SB_0.0_copy_1_2_image_25x25 "Sensitivity: 0.0, Resolution: 25x25, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_25x25_copy_1.npy SB_1.0_copy_1_2_image_25x25 "Sensitivity: 1.0, Resolution: 25x25, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_25x25_copy_1.npy SB_2.0_copy_1_2_image_25x25 "Sensitivity: 2.0, Resolution: 25x25, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_25x25_copy_1.npy SB_3.0_copy_1_2_image_25x25 "Sensitivity: 3.0, Resolution: 25x25, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_25x25_copy_1.npy SB_4.0_copy_1_2_image_25x25 "Sensitivity: 4.0, Resolution: 25x25, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_25x25_copy_1.npy SB_5.0_copy_1_2_image_25x25 "Sensitivity: 5.0, Resolution: 25x25, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_25x25_copy_1.n

In [4]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_25x25_aug_1.npy'
    model_name = f'SB_{i:.1f}_aug_1_2_image_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Augmentation: 1'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_25x25_aug_1.npy SB_0.0_aug_1_2_image_25x25 "Sensitivity: 0.0, Resolution: 25x25, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_25x25_aug_1.npy SB_1.0_aug_1_2_image_25x25 "Sensitivity: 1.0, Resolution: 25x25, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_25x25_aug_1.npy SB_2.0_aug_1_2_image_25x25 "Sensitivity: 2.0, Resolution: 25x25, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_25x25_aug_1.npy SB_3.0_aug_1_2_image_25x25 "Sensitivity: 3.0, Resolution: 25x25, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_25x25_aug_1.npy SB_4.0_aug_1_2_image_25x25 "Sensitivity: 4.0, Resolution: 25x25, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_25x25_aug_1.npy SB_5.0_aug_1_2_image_25x25 "Sensitivity: 5.0, Resolution: 25x25, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/da

In [2]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_25x25_x2.npy'
    model_name = f'SB_{i:.1f}_x2_2_image_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Luminosity: x2'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_25x25_x2.npy SB_0.0_x2_2_image_25x25 "Sensitivity: 0.0, Resolution: 25x25, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_25x25_x2.npy SB_1.0_x2_2_image_25x25 "Sensitivity: 1.0, Resolution: 25x25, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_25x25_x2.npy SB_2.0_x2_2_image_25x25 "Sensitivity: 2.0, Resolution: 25x25, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_25x25_x2.npy SB_3.0_x2_2_image_25x25 "Sensitivity: 3.0, Resolution: 25x25, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_25x25_x2.npy SB_4.0_x2_2_image_25x25 "Sensitivity: 4.0, Resolution: 25x25, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_25x25_x2.npy SB_5.0_x2_2_image_25x25 "Sensitivity: 5.0, Resolution: 25x25, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_25x25_x2.npy SB_6.0_

## Separate Batchnormalization layer

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_25x25.npy'
    model_name = f'SB_{i:.1f}_2_image_25x25_BN'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_25x25.npy SB_0.0_2_image_25x25_BN "Sensitivity: 0.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_25x25.npy SB_1.0_2_image_25x25_BN "Sensitivity: 1.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_25x25.npy SB_2.0_2_image_25x25_BN "Sensitivity: 2.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_25x25.npy SB_3.0_2_image_25x25_BN "Sensitivity: 3.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_25x25.npy SB_4.0_2_image_25x25_BN "Sensitivity: 4.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_25x25.npy SB_5.0_2_image_25x25_BN "Sensitivity: 5.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_25x25.npy SB_6.0_2_image_25x25_BN "Sensitivity: 6.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_25x25_copy_1.npy'
    model_name = f'SB_{i:.1f}_copy_1_2_image_25x25_BN'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Copy: 1'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_25x25_copy_1.npy SB_0.0_copy_1_2_image_25x25_BN "Sensitivity: 0.0, Resolution: 25x25, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_25x25_copy_1.npy SB_1.0_copy_1_2_image_25x25_BN "Sensitivity: 1.0, Resolution: 25x25, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_25x25_copy_1.npy SB_2.0_copy_1_2_image_25x25_BN "Sensitivity: 2.0, Resolution: 25x25, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_25x25_copy_1.npy SB_3.0_copy_1_2_image_25x25_BN "Sensitivity: 3.0, Resolution: 25x25, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_25x25_copy_1.npy SB_4.0_copy_1_2_image_25x25_BN "Sensitivity: 4.0, Resolution: 25x25, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_25x25_copy_1.npy SB_5.0_copy_1_2_image_25x25_BN "Sensitivity: 5.0, Resolution: 25x25, Copy: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_

## SB region: 4400-4700 5500-5800

In [24]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new_SB/mix_sample_{i:.1f}_25x25.npy'
    model_name = f'SB_{i:.1f}_2_image_25x25_new_SB'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, SB region: 4400-4700, 5500-5800'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new_SB/mix_sample_0.0_25x25.npy SB_0.0_2_image_25x25_new_SB "Sensitivity: 0.0, Resolution: 25x25, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/new_SB/mix_sample_1.0_25x25.npy SB_1.0_2_image_25x25_new_SB "Sensitivity: 1.0, Resolution: 25x25, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/new_SB/mix_sample_2.0_25x25.npy SB_2.0_2_image_25x25_new_SB "Sensitivity: 2.0, Resolution: 25x25, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/new_SB/mix_sample_3.0_25x25.npy SB_3.0_2_image_25x25_new_SB "Sensitivity: 3.0, Resolution: 25x25, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/new_SB/mix_sample_4.0_25x25.npy SB_4.0_2_image_25x25_new_SB "Sensitivity: 4.0, Resolution: 25x25, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/new_SB/mix_sample_5.0_25x25.npy SB_5.0_2_image_25x25_new_SB "Sensitivity: 5.0, Resoluti

In [25]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new_SB/mix_sample_{i:.1f}_25x25_copy_1.npy'
    model_name = f'SB_{i:.1f}_copy_1_2_image_25x25_new_SB'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Copy: 1, SB region: 4400-4700, 5500-5800'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new_SB/mix_sample_0.0_25x25_copy_1.npy SB_0.0_copy_1_2_image_25x25_new_SB "Sensitivity: 0.0, Resolution: 25x25, Copy: 1, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/new_SB/mix_sample_1.0_25x25_copy_1.npy SB_1.0_copy_1_2_image_25x25_new_SB "Sensitivity: 1.0, Resolution: 25x25, Copy: 1, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/new_SB/mix_sample_2.0_25x25_copy_1.npy SB_2.0_copy_1_2_image_25x25_new_SB "Sensitivity: 2.0, Resolution: 25x25, Copy: 1, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/new_SB/mix_sample_3.0_25x25_copy_1.npy SB_3.0_copy_1_2_image_25x25_new_SB "Sensitivity: 3.0, Resolution: 25x25, Copy: 1, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/new_SB/mix_sample_4.0_25x25_copy_1.npy SB_4.0_copy_1_2_image_25x25_new_SB "Sensitivity: 4.0, Resolution: 25x25, Copy: 1, SB region: 4400-4700, 5500-5800"
python train_CN

## Split validation

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_25x25.npy'
    model_name = f'SB_{i:.1f}_2_image_25x25_split_val'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, SB region: 4400-4700, 5500-5800'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_25x25.npy SB_0.0_2_image_25x25_split_val "Sensitivity: 0.0, Resolution: 25x25, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_25x25.npy SB_1.0_2_image_25x25_split_val "Sensitivity: 1.0, Resolution: 25x25, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_25x25.npy SB_2.0_2_image_25x25_split_val "Sensitivity: 2.0, Resolution: 25x25, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_25x25.npy SB_3.0_2_image_25x25_split_val "Sensitivity: 3.0, Resolution: 25x25, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_4.0_25x25.npy SB_4.0_2_image_25x25_split_val "Sensitivity: 4.0, Resolution: 25x25, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_5.0_25x25.npy SB_5.0_2_image_25x25_s

In [2]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_25x25_copy_1.npy'
    model_name = f'SB_{i:.1f}_copy_1_2_image_25x25_split_val'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Copy: 1, SB region: 4400-4700, 5500-5800'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_25x25_copy_1.npy SB_0.0_copy_1_2_image_25x25_split_val "Sensitivity: 0.0, Resolution: 25x25, Copy: 1, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_25x25_copy_1.npy SB_1.0_copy_1_2_image_25x25_split_val "Sensitivity: 1.0, Resolution: 25x25, Copy: 1, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_25x25_copy_1.npy SB_2.0_copy_1_2_image_25x25_split_val "Sensitivity: 2.0, Resolution: 25x25, Copy: 1, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_25x25_copy_1.npy SB_3.0_copy_1_2_image_25x25_split_val "Sensitivity: 3.0, Resolution: 25x25, Copy: 1, SB region: 4400-4700, 5500-5800"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_4.0_25x25_copy_1.npy SB_4.0_copy_1_2_image_25x25_split_val "Sensitivity: 4.0, Resolution: 25x25, Copy: 1, SB region: 4400-47

## New data process flow

#### Original mix dataset

In [11]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_75x75 "Sensitivity: 0.0, Resolution: 75x75"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_75x75 "Sensitivity: 1.0, Resolution: 75x75"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_75x75 "Sensitivity: 2.0, Resolution: 75x75"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_3.0_75x75 "Sensitivity: 3.0, Resolution: 75x75"


In [12]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_25x25 "Sensitivity: 0.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_25x25 "Sensitivity: 1.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_25x25 "Sensitivity: 2.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_3.0_25x25 "Sensitivity: 3.0, Resolution: 25x25"


### Copy: +1

In [14]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_copy_1_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_copy_1_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, Copy: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_copy_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_copy_1_75x75 "Sensitivity: 0.0, Resolution: 75x75, Copy: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_copy_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_copy_1_75x75 "Sensitivity: 1.0, Resolution: 75x75, Copy: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_copy_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_copy_1_75x75 "Sensitivity: 2.0, Resolution: 75x75, Copy: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_copy_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_75x75.npy ../Sample/HVmodel/data/split_va

In [15]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_copy_1_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_copy_1_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Copy: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_copy_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_copy_1_25x25 "Sensitivity: 0.0, Resolution: 25x25, Copy: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_copy_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_copy_1_25x25 "Sensitivity: 1.0, Resolution: 25x25, Copy: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_copy_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_copy_1_25x25 "Sensitivity: 2.0, Resolution: 25x25, Copy: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_copy_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/data/split_va

### Augmentation: +1, +3

In [5]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_aug_1_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_aug_1_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, Augmentation: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_aug_1_75x75 "Sensitivity: 0.0, Resolution: 75x75, Augmentation: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_aug_1_75x75 "Sensitivity: 1.0, Resolution: 75x75, Augmentation: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_aug_1_75x75 "Sensitivity: 2.0, Resolution: 75x75, Augmentation: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_75x75.npy ../Sample/HVmo

In [6]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_aug_1_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_aug_1_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Augmentation: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_aug_1_25x25 "Sensitivity: 0.0, Resolution: 25x25, Augmentation: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_aug_1_25x25 "Sensitivity: 1.0, Resolution: 25x25, Augmentation: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_aug_1_25x25 "Sensitivity: 2.0, Resolution: 25x25, Augmentation: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_25x25.npy ../Sample/HVmo

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_aug_3_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_aug_3_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, Augmentation: +3'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_aug_3_75x75 "Sensitivity: 0.0, Resolution: 75x75, Augmentation: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_aug_3_75x75 "Sensitivity: 1.0, Resolution: 75x75, Augmentation: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_aug_3_75x75 "Sensitivity: 2.0, Resolution: 75x75, Augmentation: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_75x75.npy ../Sample/HVmo

In [2]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_aug_3_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_aug_3_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Augmentation: +3'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_aug_3_25x25 "Sensitivity: 0.0, Resolution: 25x25, Augmentation: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_aug_3_25x25 "Sensitivity: 1.0, Resolution: 25x25, Augmentation: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_aug_3_25x25 "Sensitivity: 2.0, Resolution: 25x25, Augmentation: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_25x25.npy ../Sample/HVmo

## Luminosity: $\times 2$

In [18]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_x2_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_x2_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_x2_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Luminosity: x2'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_x2_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_x2_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_x2_25x25 "Sensitivity: 0.0, Resolution: 25x25, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_x2_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_x2_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_x2_25x25 "Sensitivity: 1.0, Resolution: 25x25, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_x2_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_x2_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_x2_25x25 "Sensitivity: 2.0, Resolution: 25x25, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_x2_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_x2_val_25x25.npy ../Sample/HVmodel/data/split_